# Franka Robot Grasping Tutorial: With and Without Spinning Pads

This tutorial demonstrates a comparison between two Franka Emika Panda robots—one with spinning pads and one without—performing the same grasping task. The spinning pads reduce the torsional friction about the normal vector of the contact between robot's finger and the object. 
"The implementation for friction in genesis is not as advanced that in Mujoco. There is only 1 scalar parameter in Genesis, and condim cannot be changed (only condim = 3 is implemented). There is no torsional friction at the moment. The torsional friction in genesis is the result of multi-contact friction."


The standard robot uses joints 0-8, with 0-6 for arm movement and 7-8 for the gripper. The modified robot includes spinning pads at joints 9-10, which can help provide rotational slippage at the contact point

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner


import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

[I 05/15/25 15:14:41.108 23566] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
gs.destroy()
gs.init(backend=gs.gpu)
gs.set_random_seed(seed=42)
adapter = GenesisAdapter()
planner = ScrewMotionPlanner()

[Genesis] [15:14:44] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [15:14:44] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [15:14:44] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [15:14:45] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [15:14:45] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [15:14:45] [INFO] Scene <243a364> created.


## Scene Construction

Here we create the simulation environment with:
1. A ground plane
2. Two Franka Emika Panda robots:
   - One standard robot at position (0,2,0)
   - One with spinning pads and virtual finger at position (0,0,0)
3. Two cubes for each robot
4. A target entity for visualization

The key difference between the robots is that the standard Franka `(franka_MJCF)` uses only the arm and finger joints (joints 0-8), while the other version has additional spinning pads (joints 9-10) that cause rotational slippage using extra hinge DoFs.


In [3]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka_MJCF = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml",
                   pos = (0.0, 2.0, 0.0)),
    
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda_vir.xml")
                   
)
cube_size = (0.1, 0.07, 0.25)
cube_pos = [(0.7, 0, 0.05), (0.7, 2.0, 0.05)]

cube, cube2 = [scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = pos,
        euler = (0, -90, 0)
    ),
    surface=gs.surfaces.Default(
        color=(0.5, 0.8, 0.94),
    ),
    material=gs.materials.Rigid(friction=5)
    
) for pos in cube_pos]

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

[Genesis] [15:14:46] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <c14fbb8>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [15:14:46] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <5bc8299>, morph: <gs.morphs.MJCF(file='/home/vahid/envs/panda/lib/python3.10/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [15:14:47] [WARNING] (MJCF) Tendon not supported
[Genesis] [15:14:47] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [15:14:47] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [15:14:47] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [15:14:47] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [15:14:47] [INFO] Appl

## Robot Configuration

We build the scene and define important joint groups:
- `motors_dof`: Joints 0-6 for arm movement
- `fingers_dof`: Joints 7-8 for gripper fingers
- `virtual_hand`: Joint 9 at the middle of the finger for rotational slippage (only on the `franka`)
- `spinning_pads`: Joints 9-10 for the spinning contact pads (only on the `franka`)

In [4]:
n_envs = 0
scene.build()


[Genesis] [15:14:51] [INFO] Building scene <243a364>...


[Genesis] [15:14:52] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [15:14:52] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 05/15/25 15:14:54.079 23566] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [15:15:00] [INFO] Compiling simulation kernels...
[Genesis] [15:18:44] [INFO] Building visualizer...
[Genesis] [15:18:48] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:
for joint in franka.joints:
    print(joint.name, joint.idx_local)
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)
virtual_finger = 9
spinning_pads = np.arange(10, 12)

joint1 0
joint2 1
joint3 2
joint4 3
joint5 4
joint6 5
joint7 6
finger_joint1 7
finger_joint2 8
virtual_finger_joint 9
left_pad_hinge 10
right_pad_hinge 11


## Initial Robot Positioning

Reset the scene and set both robots to the same initial joint configuration. We'll use this "ready pose" as a starting position for our task. Notice that the standard robot has 9 DOFs (degrees of freedom), while the modified version can have 1 dof extra for virtual_finger and two extras for spinning_pads

In [124]:
scene.reset()
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

# set joints position
franka.ready_qpos = np.append(qr, [0, 0, 0])
franka.set_qpos(franka.ready_qpos)
# position control
franka.control_dofs_position(franka.ready_qpos)

franka_MJCF.ready_qpos = qr
franka_MJCF.set_qpos(franka_MJCF.ready_qpos)
franka_MJCF.control_dofs_position(franka_MJCF.ready_qpos)

end_effector = franka.get_link('hand')
virtual_finger = franka.get_link('virtual_finger')

end_effector_MJCF = franka_MJCF.get_link('hand')


target_entity.set_pos(virtual_finger.get_pos())
target_entity.set_quat(virtual_finger.get_quat())


scene.step()

[Genesis] [18:33:58] [INFO] Resetting Scene <243a364>.


In [125]:
wPQb = adapter.get_pose(cube)
l, w, h, = adapter.get_size(cube)
bTg    = sm.SE3(l/2 - 0.02, 0, h/2 - 0.02).A
wTg = adapter.transform(cube, bTg, apply=False)
adapter.set_pose(target_entity, wTg)
scene.step()



In [126]:
adapter.step_simulation(1)

## Pre-grasp Positioning

Now we'll move both robots to a position above the cube. We'll use inverse kinematics to calculate the joint positions needed to place the end-effector at the desired position and orientation. 


In [127]:
wPQg = adapter.to(wTg, output_type='pq')
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = wPQg[:3],
    
)
qpos[fingers_dof] = 0.04


In [128]:
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 250, # 2s duration
)

[Genesis] [18:34:01] [INFO] Path solution found successfully.


## Path Planning

Generate a smooth trajectory for each robot to reach the pre-grasp position.

In [129]:
# execute the planned path
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()


## Reaching for the Object

Calculate and execute a path to move from the pre-grasp position down to the actual grasp position. Both robots move their end-effectors to the side of their respective cubes.


In [130]:
franka.control_dofs_force(np.array([-2, -2]), fingers_dof)
adapter.step_simulation(1)

In [131]:
virtual_finger_pose = adapter.forward_kinematics(franka, virtual_finger)

cube_bbox = adapter.get_bbox(cube)
qs, s_axes = planner.screw_from_bbox(cube_bbox)
q, s_axis = qs[1], s_axes[1]


finger_pqs = planner.generate_screw_trajectory(initial_pose=virtual_finger_pose, q=q, s_axis=s_axis, output_type='pq')

qposs = []
for pq in finger_pqs:
    qpos = franka.inverse_kinematics(
        link = virtual_finger,
        pos = pq[:3],
        init_qpos = qpos        
    )
    qposs.append(qpos)

In [132]:

for waypoint in qposs:
    franka.control_dofs_position(waypoint[motors_dof], motors_dof)
    target_entity.set_pos(virtual_finger.get_pos())
    target_entity.set_quat(virtual_finger.get_quat())
    scene.step()



In [133]:
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
adapter.step_simulation(1)